# Anomoly Detection of Fetch Data:

Files in .csv format of nodes, edges and labels are used for finding the anomoly detection in the Fetch provided data.
The data is analysed initially with pytorch library with dependencies in GPU mode run time.
Installing necessary libraries

In [ ]:
!pip install torchtext
!pip install torchaudio
!pip install fastai

!pip uninstall torch -y
!pip uninstall torchvision -y
!pip install torch
!pip install torchvision

# If something breaks in the notebook it is probably related to a mismatch between the Python version, CUDA or torch
import torch
pytorch_version = f"torch-{torch.__version__}.html"
!pip install --no-index torch-scatter -f https://pytorch-geometric.com/whl/$pytorch_version
!pip install --no-index torch-sparse -f https://pytorch-geometric.com/whl/$pytorch_version
!pip install --no-index torch-cluster -f https://pytorch-geometric.com/whl/$pytorch_version
!pip install --no-index torch-spline-conv -f https://pytorch-geometric.com/whl/$pytorch_version
!pip install torch-geometric

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Found existing installation: torch 1.13.1+cu116
Uninstalling torch-1.13.1+cu116:
  Successfully uninstalled torch-1.13.1+cu116
Found existing installation: torchvision 0.14.1+cu116
Uninstalling torchvision-0.14.1+cu116:
  Successfully uninstalled torchvision-0.14.1+cu116
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.4/887.4 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 KB 66.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 2.8 MB/s eta 0:00:00
     ━━━

In [ ]:
!pip install pygod

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.9/54.9 KB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pygod: filename=pygod-0.3.1-py3-none-any.whl size=76492 sha256=a820dc3d248eac0077472649777ee96bc196bbcadfb4a03230d159d94634a024
  Stored in directory: /root/.cache/pip/wheels/01/21/16/7d56f68f9b8fe255f8a59a34c2a232f054491b407c56727bb8
Successfully built pygod


In [ ]:
!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-1.10.0+cu113.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-1.10.0+cu113.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


### Importing libraries for the Graph machine learning anomaly detection

In [ ]:
import torch
from torch_geometric.data import Data
from torch_geometric.utils.convert import to_networkx
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
import torch_geometric.transforms as T
from pygod.models import DOMINANT
from sklearn.metrics import roc_auc_score, average_precision_score
from pygod.models import AnomalyDAE

### Loading .csv files from github data

In [ ]:


nodes=pd.read_csv("https://raw.githubusercontent.com/SriBhanuGundu/Fetch_Data/main/data/Anamoly_Detection/nodes.csv") #nodes x

edges=pd.read_csv("https://raw.githubusercontent.com/SriBhanuGundu/Fetch_Data/main/data/Anamoly_Detection/edges.csv") #edges - edge_index

labels=pd.read_csv("https://raw.githubusercontent.com/SriBhanuGundu/Fetch_Data/main/data/Anamoly_Detection/labels.csv") #labels y



### Masking and creating data for modeling

In [ ]:
edge_idx = torch.tensor(edges.to_numpy().transpose(), dtype=torch.long)
map_dict = {v.item():i for i,v in enumerate(torch.unique(edge_idx))}
map_edge = torch.zeros_like(edge_idx)
for k,v in map_dict.items():
    map_edge[edge_idx==k] = v
map_dict, map_edge, map_edge.shape


attrs = torch.tensor(nodes.to_numpy(), dtype=torch.float)
pad = torch.zeros((attrs.shape[0], 4), dtype=torch.float)
x = torch.cat((attrs, pad), dim=-1)

edge_idx = map_edge.long()

label = labels.loc[0:5794]
np_lab = label.to_numpy()
y = torch.tensor(np_lab, dtype=torch.long)


### Data loaded using geometric data

In [ ]:
graph = Data(x=x, edge_index=edge_idx,  y=y)
graph

Data(x=[5794, 8], edge_index=[2, 430548], y=[5794, 1])

In [ ]:
type(graph)

torch_geometric.data.data.Data

## Modeling:

Splitting of data 90% and 10%

In [ ]:
split = T.RandomNodeSplit(num_val=0.1, num_test=0.2)
graph = split(graph)

In [ ]:
graph

Data(x=[5794, 8], edge_index=[2, 430548], y=[5794, 1], train_mask=[5794], val_mask=[5794], test_mask=[5794])

### creating data frame with columns score and predictions for storing outliers detected in the model after fitting split graph in the models of DOMINANT and Anamoly DAE.

In [ ]:
df1=pd.DataFrame(columns = ['score_DOM','predict_DOM','score_DAE','predict_DAE'])

Functions defined to fit the trained model and predict the model accuracy and AUC score.

In [ ]:

def train_anomaly_detector(model, graph):
    return model.fit(graph)

def eval_anomaly_detector(model, graph):

    outlier_scores = model.decision_function(graph)
    score = outlier_scores
    
    auc = roc_auc_score(graph.y.numpy(), outlier_scores)
    ap = average_precision_score(graph.y.numpy(), outlier_scores)
    print(f'AUC Score: {auc:.3f}')
    print(f'AP Score: {ap:.3f}')
    return score


### DOMINANT model is fitted and trained. Predicted outliers for the model

In [ ]:
graph.y = graph.y.bool()
model = DOMINANT(num_layers=4, epoch=500)
model = train_anomaly_detector(model, graph)
df1.score_DOM = eval_anomaly_detector(model, graph)

AUC Score: 0.574
AP Score: 0.397


In [ ]:
df1.predict_DOM = model.predict(graph,return_confidence=False)

### AnomalyDAE model is fitted and trained. Predicted outliers for the model.

In [ ]:
model = AnomalyDAE(epoch=500)
model = train_anomaly_detector(model, graph)
df1.score_DAE = eval_anomaly_detector(model, graph)

AUC Score: 0.654
AP Score: 0.315


In [ ]:
df1.predict_DAE = model.predict(graph,return_confidence=False)

### Dataframe with scores and predictions

In [ ]:
df1

,score_DOM,predict_DOM,score_DAE,predict_DAE
0,15.693908,0,33.795963,0
1,15.693907,0,33.639153,0
2,15.383542,0,43.659882,0
3,15.383541,0,43.800365,0
4,15.948600,0,43.668385,0
...,...,...,...,...
5789,12.672811,0,45.673244,0
5790,17.043716,1,31.164858,0
5791,11.103766,0,45.732697,0
5792,9.728340,0,31.386263,0


### Prediction of anomalies for DOMINANT model. '1' indicates anomaly in the data nodes

In [ ]:
df1.query('predict_DOM>0')

,score_DOM,predict_DOM,score_DAE,predict_DAE
12,19.783344,1,44.569866,0
13,18.682104,1,45.662579,0
14,18.675646,1,46.541424,0
15,20.614969,1,43.997978,0
141,19.786816,1,43.709827,0
...,...,...,...,...
5766,24.119200,1,45.598434,0
5767,21.382107,1,46.458050,1
5776,17.033789,1,31.174856,0
5780,24.089432,1,45.585361,0


Predicted 512 nodes of anomalies

### Prediction of anomalies for AnomalyDAE model. '1' indicates anomaly in the data nodes

In [ ]:
df1.query('predict_DAE>0')

,score_DOM,predict_DOM,score_DAE,predict_DAE
20,14.030141,0,33.653671,1
113,15.247418,0,47.285824,1
114,15.247418,0,47.210438,1
163,13.084849,0,35.743015,1
164,12.897535,0,37.299747,1
...,...,...,...,...
5711,20.155157,1,47.582489,1
5726,17.187220,1,46.904766,1
5743,17.533249,1,46.313496,1
5753,16.995142,1,47.005829,1


predicted 406 nodes of anomlies

In [ ]:
df1.query('predict_DAE>0 and predict_DOM>0')

,score_DOM,predict_DOM,score_DAE,predict_DAE
251,22.216036,1,45.694450,1
327,22.375750,1,46.920486,1
412,22.260862,1,46.829918,1
483,22.285364,1,46.785393,1
559,18.709158,1,46.875576,1
...,...,...,...,...
5711,20.155157,1,47.582489,1
5726,17.187220,1,46.904766,1
5743,17.533249,1,46.313496,1
5753,16.995142,1,47.005829,1


Both the models predicted 109 common nodes of anomalies.

### https://towardsdatascience.com/well-log-data-outlier-detection-with-machine-learning-a19cafc5ea37